In [2]:
import socket
import os
import numpy as np
import cv2
from pynq import Overlay
from pynq.lib.dma import DMA
from pynq import allocate

# Load the overlay and DMA
overlay = Overlay("design_1.bit")
dma = overlay.axi_dma_0

# Create a socket for the server
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('0.0.0.0', 9092))  # Listen on all available interfaces
server_socket.listen(1)

print("Waiting for a connection...")
conn, addr = server_socket.accept()
print(f"Connected by {addr}")

while True:
    # Receive image dimensions first
    image_info = conn.recv(1024).decode()
    if not image_info:
        break  # Exit the loop if no data is received

    height, width, channels = map(int, image_info.split(','))

    # Allocate buffers for input and output
    input_buffer = allocate(shape=(height * width * channels,), dtype=np.uint8)
    output_buffer = allocate(shape=(height * width * channels,), dtype=np.uint8)

    # Receive image bytes
    image_bytes = b''
    while len(image_bytes) < height * width * channels:
        packet = conn.recv(4096)
        if not packet:
            break
        image_bytes += packet

    # Copy the image bytes to the input buffer
    np.copyto(input_buffer, np.frombuffer(image_bytes, dtype=np.uint8))

    # Process the image in chunks
    rows_per_chunk = 8
    for i in range(0, height, rows_per_chunk):
        start_row = i
        end_row = min(i + rows_per_chunk, height)

        # Transfer current chunk
        dma.sendchannel.transfer(input_buffer[start_row * width * channels:end_row * width * channels])
        dma.recvchannel.transfer(output_buffer[start_row * width * channels:end_row * width * channels])
        
        # Wait for transfer to complete
        dma.sendchannel.wait()
        dma.recvchannel.wait()

    # Send the processed image back to the client
    conn.sendall(output_buffer[:height * width * channels])

# Close connections
conn.close()
server_socket.close()
print("Mission Accomplished Successfully.....")

Waiting for a connection...
Connected by ('10.21.236.238', 46410)
